In [1]:
!pip install langchain langchain_fireworks langchain_community beautifulsoup4 google-search-results chromadb langchainhub sentence-transformers langchain-chroma gradio aiolimiter lxml faiss-cpu flashrank imdbpy

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 634.2 kB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  U

In [2]:
import asyncio
import aiohttp
import os
from langchain import hub
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_fireworks import FireworksEmbeddings, ChatFireworks
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain.prompts import PromptTemplate
from langchain.schema import Document
import requests
from bs4 import BeautifulSoup
import nltk
from nltk.tokenize import sent_tokenize
import re
import io
import time
import sys
import gradio as gr
import asyncio
from typing import List, Tuple, Any
from langchain_community.vectorstores import FAISS
import numpy as np
from functools import lru_cache
import faiss
import httpx
from urllib.parse import urlparse
from langchain_core.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.retrievers import ContextualCompressionRetriever
from flashrank import Ranker, RerankRequest
import math
from langchain.output_parsers import PydanticOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain.output_parsers import RegexParser
from langchain.prompts import ChatPromptTemplate
from langchain.schema import HumanMessage
import imdb

In [3]:
# Set up API clients
os.environ['FIREWORKS_API_KEY'] = 'API_KEY'
os.environ["SERPER_API_KEY"] = 'API_KEY'


# Download NLTK data for sentence tokenization
nltk.download('punkt', quiet=True)

True

In [ ]:
class MovieRecommendation(BaseModel):
    title: str = Field(description="The title of the recommended movie")
    year: int = Field(description="The release year of the movie")
    genre: str = Field(description="The primary genre of the movie")
    reasons: List[str] = Field(description="List of reasons why this movie is recommended")

llm = ChatFireworks(model="accounts/fireworks/models/llama-v3p1-8b-instruct", temperature=0.5)

parser = PydanticOutputParser(pydantic_object=MovieRecommendation)

prompt = ChatPromptTemplate.from_template(
    "Recommend a movie based on the following preferences:\n"
    "Genre: {genre}\n"
    "Decade: {decade}\n"
    "Mood: {mood}\n\n"
    "{format_instructions}"
)

def get_movie_recommendation(genre: str, decade: str, mood: str) -> MovieRecommendation:
    formatted_prompt = prompt.format_prompt(
        genre=genre,
        decade=decade,
        mood=mood,
        format_instructions=parser.get_format_instructions()
    )

    message = HumanMessage(content=formatted_prompt.to_string())
    output = llm.invoke([message])
    recommendation = parser.parse(output.content)

    return recommendation

def gradio_movie_recommendation(genre: str, decade: str, mood: str) -> str:
    try:
        recommendation = get_movie_recommendation(genre, decade, mood)
        return f"""Recommended Movie: {recommendation.title} ({recommendation.year})
Genre: {recommendation.genre}

Reasons for recommendation:
{chr(10).join(f'- {reason}' for reason in recommendation.reasons)}"""
    except Exception as e:
        return f"An error occurred: {str(e)}"

# Define input components
genre_input = gr.Textbox(label="Genre")
decade_input = gr.Textbox(label="Decade")
mood_input = gr.Textbox(label="Mood")

# Define output component
output = gr.Textbox(label="Movie Recommendation")

# Create the Gradio interface
iface = gr.Interface(
    fn=gradio_movie_recommendation,
    inputs=[genre_input, decade_input, mood_input],
    outputs=output,
    title="Movie Recommendation App",
    description="Enter your preferences to get a personalized movie recommendation.",
    examples=[
        ["Science Fiction", "2020s", "Thriller"],
        ["Comedy", "1990s", "Lighthearted"],
        ["Drama", "1970s", "Intense"]
    ]
)

# Launch the app
if __name__ == "__main__":
    iface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://16a374505bd7fcab6c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
class MovieRecommendation(BaseModel):
    title: str = Field(description="The title of the recommended movie")
    year: int = Field(description="The release year of the movie")
    genre: str = Field(description="The primary genre of the movie")
    reasons: List[str] = Field(description="List of reasons why this movie is recommended")

llm = ChatFireworks(model="accounts/fireworks/models/llama-v3p1-8b-instruct", temperature=0)
parser = PydanticOutputParser(pydantic_object=MovieRecommendation)
ia = imdb.IMDb()

def parse_year_input(year_input: str) -> tuple:
    if '-' in year_input:
        start, end = map(int, year_input.split('-'))
        return start, end
    else:
        year = int(year_input)
        return year, year

def get_imdb_movies(genre: str, year_start: int, year_end: int, num_movies: int = 50) -> List[dict]:
    movies = []
    for year in range(year_start, year_end + 1):
        search_results = ia.get_movie_list(genre, year)
        for movie in search_results:
            if len(movies) >= num_movies:
                break
            ia.update(movie, info=['main', 'plot'])
            movies.append({
                'title': movie.get('title'),
                'year': movie.get('year'),
                'genres': movie.get('genres', []),
                'plot': movie.get('plot', [''])[0]
            })
        if len(movies) >= num_movies:
            break
    return movies

prompt = ChatPromptTemplate.from_template(
    "Based on the following preferences and movie options, recommend the best movie:\n"
    "Genre: {genre}\n"
    "Year range: {year_range}\n"
    "Mood: {mood}\n\n"
    "Movie options:\n{movie_options}\n\n"
    "Provide your recommendation in the following format:\n"
    "{format_instructions}"
)

def get_movie_recommendation(genre: str, year_input: str, mood: str) -> MovieRecommendation:
    year_start, year_end = parse_year_input(year_input)
    movies = get_imdb_movies(genre, year_start, year_end)

    movie_options = "\n".join([f"{m['title']} ({m['year']}): {m['plot'][:100]}..." for m in movies[:5]])

    formatted_prompt = prompt.format_prompt(
        genre=genre,
        year_range=f"{year_start}-{year_end}",
        mood=mood,
        movie_options=movie_options,
        format_instructions=parser.get_format_instructions()
    )

    message = HumanMessage(content=formatted_prompt.to_string())
    output = llm.invoke([message])

    recommendation = parser.parse(output.content)
    return recommendation

if __name__ == "__main__":
    recommendation = get_movie_recommendation(
        genre="Science Fiction",
        year_input="2018-2020",
        mood="Exciting"
    )
    print(f"Recommended Movie: {recommendation.title} ({recommendation.year})")
    print(f"Genre: {recommendation.genre}")
    print("Reasons for recommendation:")
    for reason in recommendation.reasons:
        print(f"- {reason}")

/usr/local/lib/python3.10/dist-packages/imdb/parser/http/__init__.py:429: UserWarning: list type not recognized make sure it starts with 'ls'
  warnings.warn("list type not recognized make sure it starts with 'ls'")


Recommended Movie: Ad Astra (2019)
Genre: Science Fiction
Reasons for recommendation:
- Exciting space exploration plot
- Well-developed characters and themes
- Visually stunning and immersive experience


In [4]:
class MovieRecommendation(BaseModel):
    title: str = Field(description="The title of the recommended movie")
    year: int = Field(description="The release year of the movie")
    genre: str = Field(description="The primary genre of the movie")
    reasons: List[str] = Field(description="List of reasons why this movie is recommended")

llm = ChatFireworks(model="accounts/fireworks/models/llama-v3p1-8b-instruct", temperature=0)
parser = PydanticOutputParser(pydantic_object=MovieRecommendation)
ia = imdb.IMDb()

def parse_year_input(year_input: str) -> tuple:
    if '-' in year_input:
        start, end = map(int, year_input.split('-'))
        return start, end
    else:
        year = int(year_input)
        return year, year

def get_imdb_movies(genre: str, year_start: int, year_end: int, num_movies: int = 50) -> List[dict]:
    movies = []
    for year in range(year_start, year_end + 1):
        search_results = ia.get_movie_list(genre, year)
        for movie in search_results:
            if len(movies) >= num_movies:
                break
            ia.update(movie, info=['main', 'plot'])
            movies.append({
                'title': movie.get('title'),
                'year': movie.get('year'),
                'genres': movie.get('genres', []),
                'plot': movie.get('plot', [''])[0]
            })
        if len(movies) >= num_movies:
            break
    return movies

prompt = ChatPromptTemplate.from_template(
    "Based on the following preferences and movie options, recommend the best movie:\n"
    "Genre: {genre}\n"
    "Year range: {year_range}\n"
    "Mood: {mood}\n\n"
    "Movie options:\n{movie_options}\n\n"
    "Provide your recommendation in the following format:\n"
    "{format_instructions}"
)

def get_movie_recommendation(genre: str, year_input: str, mood: str) -> str:
    year_start, year_end = parse_year_input(year_input)
    movies = get_imdb_movies(genre, year_start, year_end)

    movie_options = "\n".join([f"{m['title']} ({m['year']}): {m['plot'][:100]}..." for m in movies[:5]])

    formatted_prompt = prompt.format_prompt(
        genre=genre,
        year_range=f"{year_start}-{year_end}",
        mood=mood,
        movie_options=movie_options,
        format_instructions=parser.get_format_instructions()
    )

    message = HumanMessage(content=formatted_prompt.to_string())
    output = llm.invoke([message])

    recommendation = parser.parse(output.content)

    return f"""Recommended Movie: {recommendation.title} ({recommendation.year})
Genre: {recommendation.genre}
Reasons for recommendation:
{chr(10).join([f'- {reason}' for reason in recommendation.reasons])}"""

# Define Gradio interface
iface = gr.Interface(
    fn=get_movie_recommendation,
    inputs=[
        gr.Dropdown(
            choices=["Action", "Comedy", "Drama", "Science Fiction", "Thriller", "Romance", "Horror"],
            label="Genre"
        ),
        gr.Textbox(label="Year or Year Range (e.g., 2021 or 2018-2020)"),
        gr.Textbox(label="Mood (e.g., Exciting, Relaxing, Thought-provoking)")
    ],
    outputs="text",
    title="Movie Recommendation System",
    description="Get personalized movie recommendations based on genre, year, and mood.",
    examples=[
        ["Science Fiction", "2018-2020", "Exciting"],
        ["Comedy", "2021", "Uplifting"],
        ["Drama", "2015-2019", "Thought-provoking"],
        ["Action", "2020", "Thrilling"],
        ["Romance", "2017-2021", "Heartwarming"]
    ]
)

# Launch the app
iface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://539d739730f961849a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
